In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import optuna
import re
import sklearn
import math
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import lightgbm as lgb
from tqdm import tqdm
from tqdm.notebook import tqdm
from sklearn import ensemble
import matplotlib.pyplot as plt
import optuna
plt.style.use('fivethirtyeight') 

from sklearn.ensemble import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, RandomizedSearchCV, cross_val_score, GridSearchCV, LeaveOneOut
from sklearn.preprocessing import *
from sklearn.svm import SVR
from optuna.visualization import plot_slice
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import *
from sklearn.svm import LinearSVC

# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from pylab import rcParams
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
from numpy.random import normal, seed
from scipy.stats import norm
from scipy.stats import skew, kurtosis
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from optuna.integration import XGBoostPruningCallback

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.



In [2]:
DATA_PATH = '../input/amp-parkinsons-disease-progression-prediction/'
# targets = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

In [3]:
proteins = pd.read_csv(DATA_PATH + "train_proteins.csv")
proteins_test = pd.read_csv(DATA_PATH + "example_test_files/test_proteins.csv")
peptides = pd.read_csv(DATA_PATH + "train_peptides.csv")
peptides_test = pd.read_csv(DATA_PATH + "example_test_files/test_peptides.csv")
clinical =  pd.read_csv(DATA_PATH + "train_clinical_data.csv")
test = pd.read_csv(DATA_PATH + "/example_test_files/test.csv")
sample= pd.read_csv(DATA_PATH + "example_test_files/sample_submission.csv")

# 1. Define Metric SMAPE

In [4]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [5]:
# 20230331追加
def smape_10(y_true, y_pred):
    return smape(y_true + 10, y_pred + 10)

In [6]:
# 20230330追加
import tensorflow.keras.backend as K

def smape_loss(y_true, y_pred):
    y_true = y_true + 1
    y_pred = y_pred + 1
    epsilon = 0.1
    numer = K.abs(y_pred - y_true)
    denom = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = numer / (denom/2)
    smape = tf.where(tf.math.is_nan(smape), tf.zeros_like(smape), smape)
    return smape


def calculate_smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true) + 1, np.array(y_pred) + 1
    
    numer = np.round(np.abs(y_pred-y_true),0)
    denom = np.round(np.abs(y_true) + np.abs(y_pred),0)

    return 1/len(y_true) * np.sum(np.nan_to_num(numer / (denom/2))) *100

# 2. Load Train and Sample Test Data

## 2.1 Protein data

In [7]:
print('Proteins test shape:',proteins_test.shape)
proteins_test.head()

Proteins test shape: (453, 6)


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


## 2.2 Peptides data

In [8]:
print('Peptides shape:', peptides.shape)
peptides.head()

Peptides shape: (981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [9]:
# 20230331 追加
peptides.groupby('UniProt').nunique()

,visit_id,visit_month,patient_id,Peptide,PeptideAbundance
UniProt,,,,,
O00391,764,15,223,1,763
O00533,1112,15,248,8,8278
O00584,1100,15,248,2,2054
O14498,1032,15,245,1,1032
O14773,1047,15,247,2,1700
...,...,...,...,...,...
Q9UHG2,1113,15,248,4,4083
Q9UKV8,907,15,241,1,907
Q9UNU6,761,15,219,1,759


In [10]:
print('Peptides test shape:', peptides_test.shape)
peptides_test.head()

Peptides test shape: (2057, 7)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.3,0
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.6,0
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.4,0
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.0,0
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.2,0


## 2.3 Clinical data

In [11]:
# 20230330追加
# clinical = clinical.drop('upd23b_clinical_state_on_medication',axis=1).dropna()

In [12]:
print('Clinical shape:', clinical.shape)
clinical.head()

Clinical shape: (2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [13]:
# 20230331追加
clinical['updrs_1'] = clinical['updrs_1'].fillna(clinical['updrs_1'].median())
clinical['updrs_2'] = clinical['updrs_2'].fillna(clinical['updrs_2'].median())
clinical['updrs_3'] = clinical['updrs_3'].fillna(clinical['updrs_3'].median())
clinical['upd23b_clinical_state_on_medication'] = clinical['upd23b_clinical_state_on_medication'].fillna('Missing')

In [14]:
print('Test shape:', test.shape)
test.head()

Test shape: (16, 6)


,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


# 3. Some EDA

In [15]:
proteins.groupby('visit_id').agg({'UniProt':'nunique','patient_id':'count','NPX':['min','max','mean','std']}).reset_index()

visit_id UniProt patient_id       NPX                             \
               nunique      count       min          max          mean   
0      10053_0     165        165  2497.840  269126000.0  2.856580e+06   
1     10053_12     171        171  5800.870  270030000.0  2.728871e+06   
2     10053_18     208        208  1334.110  278835000.0  2.509967e+06   
3     10138_12     217        217  2520.240  365582000.0  3.002583e+06   
4     10138_24     219        219  1436.940  396894000.0  3.068891e+06   
...        ...     ...        ...       ...          ...           ...   
1108   8699_24     216        216   756.551  346067000.0  3.064059e+06   
1109    942_12     212        212  1722.770  330558000.0  2.613298e+06   
1110    942_24     217        217  1339.150  336769000.0  2.616142e+06   
1111    942_48     216        216  1272.480  358059000.0  2.768442e+06   
1112     942_6     216        216  2491.690  290111000.0  2.476880e+06   

                    
               std  
0     2.131630e+07  
1     2.092162e+07  
2     1.969453e+07  
3     2.516170e+07  
4     2.716806e+07  
...            ...  
1108  2.409420e+07  
1109  2.295228e+07  
1110  2.312662e+07  
1111  2.460543e+07  
1112  2.002174e+07  

[1113 rows x 7 columns]

In [16]:
peptides.groupby('visit_id').agg({'UniProt':'nunique','patient_id':'count','Peptide':'nunique','PeptideAbundance': ['min','max','mean','std']}).reset_index()

visit_id UniProt patient_id Peptide PeptideAbundance               \
               nunique      count nunique              min          max   
0      10053_0     165        649     649          82.9679   66333900.0   
1     10053_12     171        633     633         128.4460   73059300.0   
2     10053_18     208        868     868         108.5000   64711200.0   
3     10138_12     217        932     932         129.0240   71652400.0   
4     10138_24     219        918     918         142.6480  123897000.0   
...        ...     ...        ...     ...              ...          ...   
1108   8699_24     216        911     911         106.9420   99846400.0   
1109    942_12     212        889     889          88.3277   70888500.0   
1110    942_24     217        910     910         108.7050   71995500.0   
1111    942_48     216        907     907         148.1360   70658500.0   
1112     942_6     216        918     918         158.8690   58472500.0   

                                   
               mean           std  
0     726248.393431  3.535602e+06  
1     737183.385744  3.799654e+06  
2     601466.784320  3.006568e+06  
3     699099.199189  3.379573e+06  
4     732120.888877  4.912602e+06  
...             ...           ...  
1108  726494.824901  4.080307e+06  
1109  623193.979635  3.362987e+06  
1110  623849.652027  3.294163e+06  
1111  659297.802601  3.359265e+06  
1112  582795.558517  2.734257e+06  

[1113 rows x 8 columns]

# 4. Make dataset for training

## 4.1 Training only first month (0's visit_month)

In [17]:
df_0 = clinical[(clinical.visit_month == 0)][['visit_id','updrs_1']]
print('Train shape:', df_0.shape)
df_0.head()

Train shape: (248, 2)


,visit_id,updrs_1
0,55_0,10.0
13,942_0,3.0
28,1517_0,11.0
38,1923_0,2.0
45,2660_0,2.0


## 4.2 Feature Engineering

### 4.2.1 Proteins features

In [18]:
proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                .reset_index()
proteins_npx_ft.head()

,visit_id,NPX_min,NPX_max,NPX_mean,NPX_std
0,10053_0,2497.84,269126000.0,2.856580e+06,2.131630e+07
1,10053_12,5800.87,270030000.0,2.728871e+06,2.092162e+07
2,10053_18,1334.11,278835000.0,2.509967e+06,1.969453e+07
3,10138_12,2520.24,365582000.0,3.002583e+06,2.516170e+07
4,10138_24,1436.94,396894000.0,3.068891e+06,2.716806e+07


In [19]:
df_proteins = pd.merge(proteins, df_0, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
proteins_Uniprot_updrs.head()

,UniProt,updrs_1_sum
0,O00391,4.971014
1,O00533,5.319588
2,O00584,5.286458
3,O14498,5.217877
4,O14773,5.371585


In [20]:
df_proteins = pd.merge(proteins, proteins_Uniprot_updrs, on = 'UniProt', how = 'left')
proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                          proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
proteins_UniProt_ft.head()

,visit_id,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std
0,10053_0,4.892857,5.601449,5.300548,0.077355
1,10053_12,4.816794,5.652174,5.296073,0.099055
2,10053_18,4.297619,5.652174,5.272617,0.134631
3,10138_12,4.297619,5.652174,5.263118,0.143238
4,10138_24,4.297619,5.652174,5.269522,0.137776


### 4.2.2 Peptides features

In [21]:
peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [22]:
peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                .reset_index()
peptides_PeptideAbundance_ft.head()

,visit_id,Abe_min,Abe_max,Abe_mean,Abe_std
0,10053_0,82.9679,66333900.0,726248.393431,3.535602e+06
1,10053_12,128.4460,73059300.0,737183.385744,3.799654e+06
2,10053_18,108.5000,64711200.0,601466.784320,3.006568e+06
3,10138_12,129.0240,71652400.0,699099.199189,3.379573e+06
4,10138_24,142.6480,123897000.0,732120.888877,4.912602e+06


In [23]:
df_peptides = pd.merge(peptides, df_0, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
peptides_PeptideAbundance_updrs.head()

,Peptide,updrs_1_sum
0,AADDTWEPFASGK,5.357143
1,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,5.296703
2,AAFTEC(UniMod_4)C(UniMod_4)QAADK,5.305699
3,AANEVSSADVK,5.364780
4,AATGEC(UniMod_4)TATVGKR,5.146497


In [24]:
df_peptides = pd.merge(peptides, peptides_PeptideAbundance_updrs, on = 'Peptide', how = 'left')
peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                          peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
peptides_ft

,visit_id,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,10053_0,4.878788,5.661972,5.279278,0.092880
1,10053_12,4.816794,5.661972,5.277513,0.097712
2,10053_18,4.297619,5.661972,5.265384,0.116303
3,10138_12,4.297619,5.661972,5.253513,0.126117
4,10138_24,4.297619,5.661972,5.257710,0.123452
...,...,...,...,...,...
1108,8699_24,4.572519,5.661972,5.256902,0.123395
1109,942_12,4.572519,5.661972,5.254323,0.118205
1110,942_24,4.572519,5.652174,5.255565,0.117226
1111,942_48,4.572519,5.652174,5.253489,0.119653


### 4.2.3 Put it all together

In [25]:
df_0_1 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_1']]
df_0_2 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_2']]
df_0_3 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_3']]
df_0_4 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_4']]

df_proteins = pd.merge(proteins, df_0_1, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs1 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_2, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs2 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_3, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs3 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_4, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs4 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_1, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs1 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_2, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs2 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_3, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs3 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_4, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs4 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_proteins_fts = [proteins_Uniprot_updrs1, proteins_Uniprot_updrs2, proteins_Uniprot_updrs3, proteins_Uniprot_updrs4]
df_peptides_fts = [peptides_PeptideAbundance_updrs1, peptides_PeptideAbundance_updrs2, peptides_PeptideAbundance_updrs3, peptides_PeptideAbundance_updrs4]
df_lst = [df_0_1, df_0_2, df_0_3, df_0_4]

In [26]:
# 20230403改変
def k_statistic(array, k):
    return np.sum((array - np.mean(array)) ** k) / (len(array) * np.std(array) ** k)

def features(df, proteins, peptides, classes):
    proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'),
                                                        NPX_range=('NPX', lambda x: x.max() - x.min()), NPX_skew=('NPX', skew), NPX_kurt=('NPX', kurtosis),
                                                        NPX_k_stat_3=('NPX', lambda x: k_statistic(x, 3)), NPX_percentile_25=('NPX', lambda x: np.percentile(x, 25)),
                                                        NPX_percentile_50=('NPX', lambda x: np.percentile(x, 50)), NPX_percentile_75=('NPX', lambda x: np.percentile(x, 75)),
                                                        NPX_quartiles=('NPX', lambda x: np.percentile(x, 75) - np.percentile(x, 25)))\
                    .reset_index()
    peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),
                                                                    Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'),
                                                                    Abe_range=('PeptideAbundance', lambda x: x.max() - x.min()), Abe_skew=('PeptideAbundance', skew),
                                                                    Abe_kurt=('PeptideAbundance', kurtosis), Abe_k_stat_3=('PeptideAbundance', lambda x: k_statistic(x, 3)),
                                                                    Abe_percentile_25=('PeptideAbundance', lambda x: np.percentile(x, 25)),
                                                                    Abe_percentile_50=('PeptideAbundance', lambda x: np.percentile(x, 50)),
                                                                    Abe_percentile_75=('PeptideAbundance', lambda x: np.percentile(x, 75)),
                                                                    Abe_quartiles=('PeptideAbundance', lambda x: np.percentile(x, 75) - np.percentile(x, 25)))\
                    .reset_index()

    df_proteins = pd.merge(proteins, df_proteins_fts[classes], on = 'UniProt', how = 'left')
    proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),
                                                              proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()
    df_peptides = pd.merge(peptides, df_peptides_fts[classes], on = 'Peptide', how = 'left')
    peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),
                                                              peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()

    df = pd.merge(df, proteins_npx_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_PeptideAbundance_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, proteins_UniProt_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_ft, on = 'visit_id', how = 'left')
    df = df.fillna(df.mean())
    return df

In [27]:
train_0 = features(df_0_1, proteins, peptides, 0)
train_0

,visit_id,updrs_1,NPX_min,NPX_max,NPX_mean,NPX_std,NPX_range,NPX_skew,NPX_kurt,NPX_k_stat_3,...,Abe_percentile_75,Abe_quartiles,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,55_3,10.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
1,942_3,7.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
2,3636_3,4.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
3,4161_3,1.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
4,5645_3,5.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,62723_3,7.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
111,62792_3,0.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
112,64669_3,15.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
113,64674_3,5.0,507.771,201446000.0,2.123204e+06,1.448782e+07,2.014455e+08,12.666522,169.710762,12.666522,...,194600.0,168474.125,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0


# 5. Training

In [28]:
import warnings

warnings.filterwarnings('ignore')

In [29]:
# 20230324追加
# Set the total value 
bar = tqdm(total = 100)
# Add description
bar.set_description('Progress rate')
for i in range(100):
    # Set the progress
    bar.update(5)
    time.sleep(1)

  0%|          | 0/100 [00:00<?, ?it/s]

# 20230403 メモ

ハイパーパラメータチューニング改善
SVC: C gammma class_weight
ロジスティック回帰：　penalty l１　l２　
 　　　　　　　　　　　　　　　　　　　　　　　　　　　C

In [30]:
model = {}
mms = MinMaxScaler()
n_estimators = [5,20,50,100,1500] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 20)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10]  # minimum sample number to split a node
min_samples_leaf =[1, 3, 4] # minimum sample number that can be stored in a leaf node
depth = list(range(2, 16))
min_samples_split = list(range(5,25))
bootstrap = [True, False] # method used to sample data points


hyperparameter_grid = {
    'regressor__n_estimators': [100, 500, 1000, 2000],
    'regressor__max_depth': [3, 6, 9, 12],
    'regressor__learning_rate': [0.01, 0.03, 0.05, 0.1],
    'regressor__min_child_weight': [1,2,3],
    'regressor__subsample' : [0.5,0.8,0.9],
    'regressor__eval_metric' : ['rmse']
}

for i in range(3):
    print('--------------------------------------------------------')
    print('Model {0}'.format(i + 1))
    train_0 = features(df_lst[i], proteins, peptides, i)
    # scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'NPX_range', 'NPX_skew', 'NPX_kurt', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std', 'Abe_range', 'Abe_skew', 'Abe_kurt']
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std','NPX_range','NPX_skew','NPX_kurt','NPX_percentile_25','NPX_percentile_50','NPX_percentile_75','NPX_quartiles','Abe_min','Abe_max','Abe_mean','Abe_std','Abe_range','Abe_skew','Abe_kurt','Abe_k_stat_3','Abe_percentile_25','Abe_percentile_50','Abe_percentile_75']
    train_0[scale_col] = mms.fit_transform(train_0[scale_col])
    
    rfc = RandomForestRegressor()
    sv = SVR()
    ln = Lasso()
    dt = DecisionTreeRegressor()
    lgbm = lgb.LGBMRegressor(n_jobs=-1)
    
    svc_params = [{'kernel':['poly','rbf','sigmoid','linear']}]
    forest_params = [{'n_estimators': n_estimators,

    'max_features': max_features,

    'max_depth': max_depth,

    'min_samples_split': min_samples_split,

    'min_samples_leaf': min_samples_leaf,

    'bootstrap': bootstrap}]
    linear_params = [{'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
    
    tree_params = [{

    'max_features': max_features,

    'max_depth': max_depth,

    'min_samples_split': min_samples_split,

    'min_samples_leaf': min_samples_leaf}]
    
    param_lgb = [{
        'n_estimators': n_estimators,
        'learning_rate': [0.0005,0.01, 0.03, 0.05, 0.1],
        'max_depth': max_depth,
        'num_leaves': list(range(2, 151))
    }]
       
    # Xとyを定義します
    X = train_0.drop(columns=['visit_id', 'updrs_{0}'.format(i + 1)], axis=1)
    y = train_0['updrs_{0}'.format(i + 1)].astype(np.float32)
       
    # GroupKFoldを使用してCVを設定します
    groups = train_0['visit_id']
    cv = GroupKFold(n_splits=10)
    
    clf = RandomizedSearchCV(rfc, forest_params, cv=cv.split(X, y, groups), scoring=make_scorer(smape), verbose = -1)
    svc = RandomizedSearchCV(sv, svc_params, cv=cv.split(X, y, groups), scoring=make_scorer(smape), verbose = -1)
    linear = RandomizedSearchCV(ln, linear_params, cv=cv.split(X, y, groups), scoring=make_scorer(smape), verbose = -1)
    dec = RandomizedSearchCV(dt, tree_params, cv=cv.split(X, y, groups), scoring=make_scorer(smape), verbose = -1)
    light = RandomizedSearchCV(lgbm, param_lgb, cv=cv.split(X, y, groups), scoring=make_scorer(smape), verbose=-1)

    X = train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1)
    y = train_0['updrs_{0}'.format(i + 1)].astype(np.float32)
    
    clf.fit(X, y)
    svc.fit(X,y)
    linear.fit(X,y)
    dec.fit(X,y)
    light.fit(X, y)

    print('rfc best params:',clf.best_params_)

    print('rfc best score',clf.best_score_)
    
    print('svc best param',svc.best_params_)

    print('svc best score',svc.best_score_)
    
    print('linear best param',linear.best_params_)

    print('linear best score',linear.best_score_)
    
    print('decision tree best param',dec.best_params_)

    print('decision tree best score',dec.best_score_)
    
    
    print('Train smape rfc:',smape(train_0['updrs_{0}'.format(i + 1)], clf.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape svc:',smape(train_0['updrs_{0}'.format(i + 1)], svc.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape linear:',smape(train_0['updrs_{0}'.format(i + 1)], linear.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape decision tree:',smape(train_0['updrs_{0}'.format(i + 1)], dec.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape LightGBM:', smape(train_0['updrs_{0}'.format(i + 1)], light.predict(train_0.drop(columns=['visit_id', 'updrs_{0}'.format(i + 1)], axis=1))))
   
    # 2023033１追加
    print('Train smape_10 rfc:',smape_10(train_0['updrs_{0}'.format(i + 1)], clf.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_10 svc:',smape_10(train_0['updrs_{0}'.format(i + 1)], svc.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_10 linear:',smape_10(train_0['updrs_{0}'.format(i + 1)], linear.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_10 decision tree:',smape_10(train_0['updrs_{0}'.format(i + 1)], dec.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_10 LightGBM:', smape_10(train_0['updrs_{0}'.format(i + 1)], light.predict(train_0.drop(columns=['visit_id', 'updrs_{0}'.format(i + 1)], axis=1))))
   

    # 20230330追加
    print('Train smape_loss rfc:', smape_loss(train_0['updrs_{0}'.format(i + 1)], clf.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_loss svc:', smape_loss(train_0['updrs_{0}'.format(i + 1)], svc.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_loss linear:', smape_loss(train_0['updrs_{0}'.format(i + 1)], linear.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_loss decision tree:', smape_loss(train_0['updrs_{0}'.format(i + 1)], dec.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape_loss lgb:', smape_loss(train_0['updrs_{0}'.format(i + 1)], light.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train calculate_smape rfc:', calculate_smape(train_0['updrs_{0}'.format(i + 1)], clf.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train calculate_smape svc:', calculate_smape(train_0['updrs_{0}'.format(i + 1)], svc.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train calculate_smape linear:', calculate_smape(train_0['updrs_{0}'.format(i + 1)], linear.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train calculate_smape decision tree:', calculate_smape(train_0['updrs_{0}'.format(i + 1)], dec.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train calculate_smape lgb:', calculate_smape(train_0['updrs_{0}'.format(i + 1)], light.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
   

    model['rfc_' + str(i)] = clf
    model['svc_' + str(i)] = svc
    model['linear_' + str(i)] = linear
    model['dec_' + str(i)] = dec
    model['light_' + str(i)] = light

--------------------------------------------------------
Model 1
rfc best params: {'n_estimators': 5, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'auto', 'max_depth': 21, 'bootstrap': True}
rfc best score 71.21729823167658
svc best param {'kernel': 'linear'}
svc best score 70.73845177189487
linear best param {'alpha': 0.01}
linear best score 71.07190137126625
decision tree best param {'min_samples_split': 18, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 50}
decision tree best score 71.07190137126625
Train smape rfc: 70.5239552326753
Train smape svc: 70.49686773183348
Train smape linear: 70.46046849710056
Train smape decision tree: 70.46046849710056
Train smape LightGBM: 70.46046849710056
Train smape_10 rfc: 22.1818608133698
Train smape_10 svc: 21.085678531372682
Train smape_10 linear: 22.12375606272425
Train smape_10 decision tree: 22.12375606272425
Train smape_10 LightGBM: 22.12375606272425
Train smape_loss rfc: tf.Tensor(
[0.5035709  0.19723491 0.26

# 6. Inference

In [31]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

In [32]:
def map_test(x):
    updrs = x.split('_')[2] + '_' + x.split('_')[3]
    month = int(x.split('_plus_')[1].split('_')[0])
    visit_id = x.split('_')[0] + '_' + x.split('_')[1]
    # set all predictions 0 where updrs equals 'updrs_4'
    if updrs=='updrs_3':
#         rating = updrs_3_pred[month]
        rating = df[df.visit_id == visit_id]['pred2'].values[0]
    elif updrs=='updrs_4':
        rating = 0
    elif updrs =='updrs_1':
        rating = df[df.visit_id == visit_id]['pred0'].values[0]
    else:
        rating = df[df.visit_id == visit_id]['pred1'].values[0]
    return rating

counter = 0
# The API will deliver four dataframes in this specific order:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    df = test[['visit_id']].drop_duplicates('visit_id')
    pred_0 = features(df[['visit_id']], test_proteins, test_peptides, 0)
    # scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'NPX_range', 'NPX_skew', 'NPX_kurt', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std', 'Abe_range', 'Abe_skew', 'Abe_kurt']
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std','NPX_range','NPX_skew','NPX_kurt','NPX_percentile_25','NPX_percentile_50','NPX_percentile_75','NPX_quartiles','Abe_min','Abe_max','Abe_mean','Abe_std','Abe_range','Abe_skew','Abe_kurt','Abe_k_stat_3','Abe_percentile_25','Abe_percentile_50','Abe_percentile_75']
    
    
    pred_0[scale_col] = mms.fit_transform(pred_0[scale_col])
    pred_0 = (model['rfc_0'].predict(pred_0.drop(columns = ['visit_id'], axis = 1)) \
              + model['svc_0'].predict(pred_0.drop(columns = ['visit_id'], axis = 1))\
              + model['linear_0'].predict(pred_0.drop(columns = ['visit_id'], axis = 1))\
              + model['light_0'].predict(pred_0.drop(columns = ['visit_id'], axis = 1)))/4
    df['pred0'] = np.ceil(pred_0 + 0)

    pred_1 = features(df[['visit_id']], test_proteins, test_peptides, 1)
    # scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'NPX_range', 'NPX_skew', 'NPX_kurt', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std', 'Abe_range', 'Abe_skew', 'Abe_kurt']
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std','NPX_range','NPX_skew','NPX_kurt','NPX_percentile_25','NPX_percentile_50','NPX_percentile_75','NPX_quartiles','Abe_min','Abe_max','Abe_mean','Abe_std','Abe_range','Abe_skew','Abe_kurt','Abe_k_stat_3','Abe_percentile_25','Abe_percentile_50','Abe_percentile_75']
    
    
    pred_1[scale_col] = mms.fit_transform(pred_1[scale_col])
    pred_1 = (model['rfc_1'].predict(pred_1.drop(columns = ['visit_id'], axis = 1)) \
              + model['svc_1'].predict(pred_1.drop(columns = ['visit_id'], axis = 1))\
              + model['linear_1'].predict(pred_1.drop(columns = ['visit_id'], axis = 1))\
              + model['light_1'].predict(pred_1.drop(columns = ['visit_id'], axis = 1)))/4
    df['pred1'] = np.ceil(pred_1 + 0.5)
    
    pred_2 = features(df[['visit_id']], test_proteins, test_peptides, 2)
    # scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'NPX_range', 'NPX_skew', 'NPX_kurt', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std', 'Abe_range', 'Abe_skew', 'Abe_kurt']
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std','NPX_range','NPX_skew','NPX_kurt','NPX_percentile_25','NPX_percentile_50','NPX_percentile_75','NPX_quartiles','Abe_min','Abe_max','Abe_mean','Abe_std','Abe_range','Abe_skew','Abe_kurt','Abe_k_stat_3','Abe_percentile_25','Abe_percentile_50','Abe_percentile_75']
    
    pred_2[scale_col] = mms.fit_transform(pred_2[scale_col])
    pred_2 = (model['rfc_2'].predict(pred_2.drop(columns = ['visit_id'], axis = 1)) \
              + model['svc_2'].predict(pred_2.drop(columns = ['visit_id'], axis = 1))\
              + model['linear_2'].predict(pred_2.drop(columns = ['visit_id'], axis = 1))\
              + model['light_2'].predict(pred_2.drop(columns = ['visit_id'], axis = 1)))/4
    df['pred2'] = np.ceil(pred_2 + 1.5)
    
    sample_submission['rating'] = sample_submission['prediction_id'].apply(map_test)
    env.predict(sample_submission)
    
    if counter == 0:
        display(test)
        display(sample_submission)
        
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_0,0,3342,updrs_1,3342_0_updrs_1
1,3342_0,0,3342,updrs_2,3342_0_updrs_2
2,3342_0,0,3342,updrs_3,3342_0_updrs_3
3,3342_0,0,3342,updrs_4,3342_0_updrs_4
4,50423_0,0,50423,updrs_1,50423_0_updrs_1
5,50423_0,0,50423,updrs_2,50423_0_updrs_2
6,50423_0,0,50423,updrs_3,50423_0_updrs_3
7,50423_0,0,50423,updrs_4,50423_0_updrs_4


,prediction_id,rating
0,3342_0_updrs_1_plus_0_months,6.0
1,3342_0_updrs_1_plus_6_months,6.0
2,3342_0_updrs_1_plus_12_months,6.0
3,3342_0_updrs_1_plus_24_months,6.0
4,3342_0_updrs_2_plus_0_months,7.0
5,3342_0_updrs_2_plus_6_months,7.0
6,3342_0_updrs_2_plus_12_months,7.0
7,3342_0_updrs_2_plus_24_months,7.0
8,3342_0_updrs_3_plus_0_months,22.0
9,3342_0_updrs_3_plus_6_months,22.0


In [33]:
# 20230324追加
# Set the total value 
bar = tqdm(total = 100)
# Add description
bar.set_description('Progress rate')
for i in range(100):
    # Set the progress
    bar.update(5)
    time.sleep(1)

  0%|          | 0/100 [00:00<?, ?it/s]